# EDA (Exploratory Data Analysis)
* Dataset from the [CanYouReadIt?](https://canyoureadit.com) project
* The survey was conducted starting from **/4/20/2021.** This is an ongoing survey, but the current data has been exported from the online database at **1/8/2022.**

In [ ]:
try:
    import pandas as pd
    print(f"Pandas version: {pd.__version__} found!")
except ModuleNotFoundError:
    print("Install Pandas with: conda install pandas")